In [2]:
import pygame
import numpy as np
import sys
import importlib
from typing import Dict, Union, Tuple, Callable

import src.quaternion as q
import src.simulation as sim
import src.transformations as t
import src.coordinates as c
importlib.reload(q)
importlib.reload(sim)
importlib.reload(t)

pygame 2.5.2 (SDL 2.28.3, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


<module 'src.transformations' from '/Users/alfred/Documents/rotations/src/transformations.py'>

In [ ]:
# TODO: get quaternion of orientation 

In [5]:
def start_environment(screen_width: Union[int, float],
                      screen_height: Union[int, float],
                      camera_location: np.ndarray,
                      translation_controls: Dict[int, Tuple[Callable, Tuple]],
                      rotation_controls: Dict[int, Tuple[Callable, Tuple]],
                      uniform_scale_controls: Dict[int, Tuple[Callable, Tuple]],
                      vertices: np.ndarray,
                      faces: np.ndarray) -> None:
    assert len(vertices.shape) == 2
    assert vertices.shape[1] == 3
    assert len(faces.shape) == 2
    assert faces.shape[1] == 4

    pygame.init()

    screen = pygame.display.set_mode((screen_width, screen_height))
    pygame.display.set_caption('Affine Transformations')

    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        if pygame.key.get_pressed()[pygame.K_LSHIFT] or pygame.key.get_pressed()[pygame.K_RSHIFT]:
            sim.control(translation_controls)

        else:
            sim.control(rotation_controls)
            sim.control(uniform_scale_controls)

        sim.render(screen, vertices, faces, screen_height, sim.projection(0, camera_location))

    pygame.quit()
    sys.exit(0)


def run() -> None:
    center = np.array([300., 300., 300.])
    vertices, faces = sim.init_cuboid(center, 50, 50, 50)
    keyframes = np.full_like(vertices[:2, :], np.nan)

    translation_controls = {pygame.K_LEFT: (t.rotation_about_y, (center, vertices, - np.pi / 80)),
                            pygame.K_RIGHT: (t.rotation_about_y, (center, vertices, np.pi / 80)),
                            pygame.K_UP: (t.rotation_about_x, (center, vertices, - np.pi / 80)),
                            pygame.K_DOWN: (t.rotation_about_x, (center, vertices, np.pi / 80)),
                            pygame.K_a: (t.rotation_about_z, (center, vertices, - np.pi / 80)),
                            pygame.K_d: (t.rotation_about_z, (center, vertices, np.pi / 80))}

    rotation_controls = {pygame.K_LEFT: (t.translation, (center, vertices, - 5, 0)),
                         pygame.K_RIGHT: (t.translation, (center, vertices, 5, 0)),
                         pygame.K_UP: (t.translation, (center, vertices, 5, 1)),
                         pygame.K_DOWN: (t.translation, (center, vertices, - 5, 1)),
                         pygame.K_w: (t.translation, (center, vertices, 5, 2)),
                         pygame.K_s: (t.translation, (center, vertices, - 5, 2))}

    uniform_scale_controls = {pygame.K_n: (t.uniform_scale, (center, vertices, 0.9)),
                              pygame.K_m: (t.uniform_scale, (center, vertices, 1.1))}
    
    keyframe_record_controls = {pygame.K_i: (c.local_coordinates, (vertices[0], center, keyframes[0, :])), pygame.K_o: (c.local_coordinates, (vertices[0], center, keyframes[1, :]))}
    
    interpolate_controls = {pygame.K_p: (animate_inbetweens, (,))}

    start_environment(800, 800, np.array([400., 400., -100.]),
                      translation_controls,
                      rotation_controls,
                      uniform_scale_controls,
                      vertices, faces)

In [4]:
center = np.array([300., 300., 300.])
vertices, faces = sim.init_cuboid(center, 50, 50, 50)
keyframes = np.full_like(vertices[:2, :], np.nan)

In [5]:
keyframes

(array([[nan, nan, nan],
        [nan, nan, nan]]),
 array([[nan, nan, nan],
        [nan, nan, nan]]))

In [21]:
np.all(np.isnan(keyframes)), np.isnan(keyframes)

(True,
 array([[ True,  True,  True],
        [ True,  True,  True]]))

In [22]:
keyframe_copy = keyframes[0, :]
keyframe_copy[:] = vertices[0] - center
keyframe_copy, keyframes

(array([50., 50., 50.]),
 array([[50., 50., 50.],
        [nan, nan, nan]]))